In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
%matplotlib inline
import pandas as pd
import numpy as np 

PATH = 'movie_metadata.csv'
BIN_DURATION = 20
THRESHOLD_S_IPW = 15
THRESHOLD_S_DR = 30
YEAR_DUMP = 1980
DURATION_LOW = 80
DURATION_HIGH = 190
STRATIFICATION_BINS = 5

def get_genres(data):
    genre_set = set()
    for index,row in data.iterrows():
        genre_set.update(row['genres'].split('|'))
    return genre_set

def adjust_budget_gross(data):
  inflation = pd.read_csv('inflation_data.csv')
  inflation['CPI_Multiplier'] = inflation['CPIAUCNS'].iloc[-1] / inflation['CPIAUCNS']

  inflation['YEAR'] = inflation['DATE'].apply(lambda x: int(x[:4]))
  data['YEAR'] = data['title_year'].apply(lambda x: int(x))

  data = data.join(inflation.set_index('YEAR'), how='left', on='YEAR',lsuffix='_caller', rsuffix='_other')
  data = data.drop(columns=['YEAR','DATE','CPIAUCNS'])

  data['CPIAdjBudget'] = data['budget']*data['CPI_Multiplier']
  #data['CPIAdjGross'] = data['gross']*data['CPI_Multiplier']
  data['log_CPIAdjBudget'] = np.log(data['CPIAdjBudget'])
  #data['log_CPIAdjGross'] = np.log(data['CPIAdjGross'])

  #data['log_CPIAdjProfit'] = data['log_CPIAdjGross']-data['log_CPIAdjBudget']
  #data['log_CPIAdjROI'] = (data['log_CPIAdjGross']-data['log_CPIAdjBudget'])/data['log_CPIAdjBudget']
  
  data = data.drop(columns=['budget','CPIAdjBudget','CPI_Multiplier'],axis=1)
  return data


def categorize_content_rating(data):
  data['content_rating_R'] = np.where(data['content_rating'] == 'R',1,0)
  data['content_rating_PG_13'] = np.where(data['content_rating'] == 'PG-13',1,0)
  data['content_rating_PG'] = np.where(data['content_rating'] == 'PG',1,0)
  data['content_rating_other'] = np.where((data['content_rating'] != 'PG')
                                     & (data['content_rating'] != 'PG-13')
                                     & (data['content_rating'] != 'R'),1,0)
  
  data = data.drop(columns=['content_rating'],axis=1)
  return data
 
def filter_and_categorize_duration(data,bin_duration):
  data = data[(data['duration']>=DURATION_LOW) & (data['duration']<=DURATION_HIGH)]
  bins = [i for i in range(DURATION_LOW-1,DURATION_HIGH+1,bin_duration)]
  data['bin'],bins = pd.cut(data['duration'],bins,labels=[i for i in range(1,len(bins))],retbins=True)  
  return data, bins
  
  
def filter_year(data):
  data = data[data['title_year']>=YEAR_DUMP]
  return data
  
def data_preperation(path,bin_duration=10): 
  data = pd.read_csv(path)
  
  data.drop_duplicates(inplace=True)
  data["movie_title"] = data["movie_title"].str.strip()
  data = data.drop(columns=['color','director_name','director_facebook_likes','actor_3_facebook_likes',
                       'actor_2_name','actor_1_facebook_likes','actor_1_name',
                       'cast_total_facebook_likes','actor_3_name','facenumber_in_poster',
                       'plot_keywords','movie_imdb_link','aspect_ratio','movie_facebook_likes',
                       'language', 'country','actor_2_facebook_likes','gross'],axis=1)  
  data.dropna(subset=['budget','duration','title_year','num_critic_for_reviews'],inplace=True)
  
  genre_set = get_genres(data)
  for genre in genre_set:
    data[genre] = np.where(data['genres'].apply(lambda x: x.find(genre) > -1),1,0)
  
  data = adjust_budget_gross(data)
  data = categorize_content_rating(data)
  data = filter_year(data)  
  data, bins = filter_and_categorize_duration(data,bin_duration)
  return data, bins


data, bins = data_preperation(PATH,BIN_DURATION)
print(data.shape)

(4115, 38)


C:\Users\noamarbe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Propensity Score

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

def propensity_score_lr_predictor(X,T):
    clf = LogisticRegression(random_state=0,max_iter=10000, solver='lbfgs',multi_class='multinomial').fit(X, T)    
    ps_score = np.asarray(clf.predict_proba(X))
    
    T_pred = clf.predict(X)
    #print('Accuracy LR: ' + str(accuracy_score(T,T_pred)))
    
    return ps_score

def propensity_score_rf_predictor(X,T):
    clf = RandomForestClassifier(n_estimators=400, max_depth=5, random_state=0).fit(X,T) 
    ps_score = np.asarray(clf.predict_proba(X))

    T_pred = clf.predict(X)
    #print('Accuracy RF: ' + str(accuracy_score(T,T_pred)))
    
    return np.asarray(clf.predict_proba(X))


# Inverse Propensity Score

In [3]:
def get_group_effect(bin_data,is_treatment=1,is_rct=1):
    T = bin_data['T']
    y = bin_data['imdb_score']
    ps = bin_data['ps_1']
    group_record_size = bin_data[bin_data['T']==is_treatment].shape[0]
    n = bin_data.shape[0]
    group_effect = -1
    
    if (group_record_size >= THRESHOLD_S_IPW):
        if is_rct:
            if is_treatment:
                group_effect = ((T*y/0.5).sum())/(group_record_size)
            else:
                group_effect = ((((1-T)*y)/0.5).sum())/(group_record_size)
        else:
            if is_treatment:
                group_effect = (((T*y)/ps).sum())/n
            else:
                group_effect = ((((1-T)*y)/(1-ps)).sum())/n   
    return group_effect
    

def inverse_propensity_score(bin_data,is_rct=1):
    treatment=1
    control=0
    
    treatment_effect = get_group_effect(bin_data,treatment,is_rct) 
    control_effect = get_group_effect(bin_data,control,is_rct)
    
    if (treatment_effect != -1 and control_effect != -1):
        ate = treatment_effect - control_effect
    else:
        ate = np.nan
    return ate
   

# Stratification

In [4]:
import copy



def stratification(bin_data):
    ate_scores = []
    bin_data = bin_data.sort_values(by=['ps_1'])
    bin_data['stratification_bin'] = pd.qcut(bin_data['ps_1'],
                                             q=STRATIFICATION_BINS,
                                             labels=[i for i in range(1,STRATIFICATION_BINS+1)],
                                             retbins=False)
    is_rct=1
    
    for bin in range(1,STRATIFICATION_BINS+1):
        inner_bin_data = copy.deepcopy(bin_data[bin_data['stratification_bin']==bin])
        ate_scores.append(inverse_propensity_score(inner_bin_data,is_rct))
    ate = np.nanmean(ate_scores)
    return ate    

# Regression

In [5]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

def ridge_linear_regression(adj_bin_data,with_ps=True):
    if with_ps:
        X = adj_bin_data.drop(columns=['imdb_score','ps_0','ps_1'])
    else:
        X = X = adj_bin_data.drop(columns=['imdb_score'])
        
    y = adj_bin_data['imdb_score']
    
    ridge = Ridge(random_state=0)
    
    clf = ridge.fit(X, y) 
    T_coef = clf.coef_[-1]
    
    y_pred = clf.predict(X)
    #y_pred_T = clf.predict(X[X["T"]==1])
    #y_pred_C = clf.predict(X[X["T"]==0])
    
    #ate = (y_pred_T-y_pred_C).mean()
    
    #print('RMSE: ' + str(math.sqrt(mean_squared_error(y,y_pred))))
    
    #t_ind,p_value = stats.ttest_ind(y,y_pred,equal_var=False)
    #print ('linear_t: ' + str (t_ind) + ' linear_p: '+ str(p_value))

    return T_coef, y_pred 
  
def ridge_non_linear_regression(adj_bin_data):
    X = adj_bin_data.drop(columns=['imdb_score','ps_0','ps_1'])
    y = adj_bin_data['imdb_score']
    
    columns = X.columns
    
    ridge = Ridge(random_state=0)
    poly = PolynomialFeatures(2)
    X_poly = poly.fit_transform(X)
    
    clf = ridge.fit(X_poly, y) 
    features_coef = dict(zip(poly.get_feature_names(columns),clf.coef_))
    T_coef_non_linear = features_coef['T']

    y_pred = clf.predict(X_poly)
    #t_ind,p_value = stats.ttest_ind(y,y_pred,equal_var=False)
    #print ('non_linear_t: ' + str (t_ind) + ' non_linear_p: '+ str(p_value))
    
    #print('RMSE_non_linear: ' + str(math.sqrt(mean_squared_error(y,y_pred))))
    return T_coef_non_linear, y_pred 
    
def ridge_non_linear_regression_robust(adj_bin_data):
    X = adj_bin_data.drop(columns=['imdb_score','ps_0','ps_1'])
    columns = X.columns
    
    poly = PolynomialFeatures(2)
    X_poly = poly.fit_transform(X)
    X_poly = pd.DataFrame(data=X_poly,columns=poly.get_feature_names(columns))
    X_poly_treated = X_poly[X_poly['T']==1]
    X_poly_control = X_poly[X_poly['T']==0]
    
    y_treated = adj_bin_data[adj_bin_data['T']==1]['imdb_score']
    y_control = adj_bin_data[adj_bin_data['T']==0]['imdb_score']
    
    ridge_treated = Ridge(random_state=0)
    ridge_control = Ridge(random_state=1)
    
    clf_treated = ridge_treated.fit(X_poly_treated, y_treated) 
    clf_control = ridge_control.fit(X_poly_control, y_control) 
    
    y_pred_treated = clf_treated.predict(X_poly)
    y_pred_control = clf_control.predict(X_poly)
    
    #t_ind_treatment,p_value_treatment = stats.ttest_ind(y_treated,y_pred_treated,equal_var=False)
    #t_ind_control,p_value_control = stats.ttest_ind(y_control,y_pred_control,equal_var=False)
    
    #print ('non_linear_robust_treatment_t: ' + str (t_ind_treatment) 
    #       + ' non_linear_robust_treatment_p: '+ str(p_value_treatment))
    #print ('non_linear_robust_control_t: ' + str (t_ind_control) 
    #       + ' non_linear_robust_control_p: '+ str(p_value_control))
    #print('\n')
    return y_pred_treated,y_pred_control 

# Doubly Robust Estimator

In [6]:
def calc_doubly_robust_sum(bin_data):
    n = bin_data.shape[0]
    T = bin_data['T']
    ps = bin_data['ps_1']
    treatment_prediction = bin_data['treated_prediction']
    control_prediction = bin_data['control_prediction']
    y = bin_data['imdb_score']
    
    left_hand_sum = ((T*y/ps)-((T-ps)/ps)*treatment_prediction).sum()
    right_hand_sum = ((((1-T)*y)/(1-ps))+((T-ps)/(1-ps))*control_prediction).sum()
    sum = (left_hand_sum - right_hand_sum)/n
    return sum

def doubly_robust_estimator(bin_data):
    n = bin_data.shape[0]
    
    bin_data_treated = bin_data[bin_data['T']==1]
    bin_data_control = bin_data[bin_data['T']==0]
    
    if (bin_data_treated.shape[0] >= THRESHOLD_S_DR and bin_data_control.shape[0] >= THRESHOLD_S_DR):  
        treated_predictions,control_predictions = ridge_non_linear_regression_robust(bin_data)                  
                          
        bin_data['treated_prediction'] = treated_predictions
        bin_data['control_prediction'] = control_predictions

        ate = calc_doubly_robust_sum(bin_data)
    else:
        ate = np.nan
    return ate

# Sensitivity Anlysis

In [41]:
%matplotlib inline
import matplotlib.patches as mpatches

def sensitivity_analysis(adj_bin_data,T_coef_linear):
    adj_bin_data['error'] = adj_bin_data['imdb_score'] - adj_bin_data['imdb_score_predicted']
    var_error_treated = adj_bin_data[adj_bin_data['T']==1]['error'].var()
    var_error_control = adj_bin_data[adj_bin_data['T']==0]['error'].var()
    
    a=2
    step=0.05
    
    lamda_values = np.arange(start=-a,stop=a+step,step=step)
    ate_values = []
    
    for lamda in lamda_values:
        rho_1 = lamda*var_error_treated
        bias_1_col = adj_bin_data['ps_0']*rho_1
        bias_1 = bias_1_col.mean()
        
        rho_0 = lamda*var_error_control
        bias_0_col = adj_bin_data['ps_1']*rho_0.mean()
        bias_0 = bias_0_col.mean()
        
        res = T_coef_linear - bias_1 - bias_0
        ate_values.append(res)
    
    index = -1
    min_value = min(np.absolute(ate_values))
    if min_value in ate_values:
        index = ate_values.index(min_value)
    else:
        index = ate_values.index(min_value*(-1))
                    
    tipping_point = lamda_values[index]
    
    
    """plt.plot(lamda_values,ate_values)
    blue_patch = mpatches.Patch(color='blue', label='ATE')
    plt.legend(handles=[blue_patch])
    plt.ylim(-2,2)
    plt.xlim(-(a+step),a+step)
    plt.axhline(y=0,linestyle=':')
    plt.show()
    plt.close()"""
    
    rho_1_f = tipping_point*var_error_treated
    rho_0_f = tipping_point*var_error_control
    #print(rho_1_f)
    #print(rho_0_f)
    return rho_0_f,rho_1_f

# Run tests

In [42]:
from sklearn.preprocessing import StandardScaler

results_dict = dict()
stratification_ate_list = list()
linear_regression_estimator_list = list()
non_linear_regression_estimator_list = list()
inverse_ps_list = list()
doubly_robust_list = list()

for bin in range(1,len(bins)-1): 
    bin_data = copy.deepcopy(data[(data['bin']==bin) | (data['bin']==bin+1)])
    
    X = bin_data.drop(columns=['imdb_score','bin','duration','movie_title','genres'],axis=1) 
    columns = X.columns
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X,columns=columns)

    bin_data['T'] = np.where(bin_data['bin']==bin,0,1)
    T = bin_data['T']
    
    y = bin_data['imdb_score']
    
    propensity_score_lr = propensity_score_lr_predictor(X,T)
    propensity_score = pd.DataFrame(data={'ps_0':propensity_score_lr[:,0],
                                          'ps_1':propensity_score_lr[:,1]},
                                   columns=['ps_0','ps_1'])   
    
    T = T.reset_index()
    propensity_score = propensity_score.reset_index()
    y = y.reset_index()
    
    adj_bin_data = pd.concat([X,propensity_score,T,y],axis=1)
    adj_bin_data = adj_bin_data.drop(columns=['index'],axis=1)

    stratification_ate_list.append(stratification(adj_bin_data))
                                                                        
    T_coef_non_linear, _ = ridge_non_linear_regression(adj_bin_data)
    T_coef_linear, y_pred = ridge_linear_regression(adj_bin_data)
    linear_regression_estimator_list.append(T_coef_linear)
    non_linear_regression_estimator_list.append(T_coef_non_linear)
    
    adj_bin_data['imdb_score_predicted'] = y_pred
    tipping_point = sensitivity_analysis(adj_bin_data,T_coef_linear)
    
    is_rct = 0
    inverse_ps_list.append(inverse_propensity_score(adj_bin_data,is_rct))
    
    doubly_robust_list.append(doubly_robust_estimator(adj_bin_data))
    
stratification_ate = np.nanmean(stratification_ate_list)
stratification_ate_list.append(stratification_ate)
results_dict['stratification'] = stratification_ate_list

linear_regression_ate = np.mean(linear_regression_estimator_list)
linear_regression_estimator_list.append(linear_regression_ate)
non_linear_regression_ate = np.mean(non_linear_regression_estimator_list)
non_linear_regression_estimator_list.append(non_linear_regression_ate)
results_dict['linear_regression'] = linear_regression_estimator_list
results_dict['non_linear_regression'] = non_linear_regression_estimator_list

inverse_ps_ate = np.nanmean(inverse_ps_list)
inverse_ps_list.append(inverse_ps_ate)
results_dict['inverse_ps'] = inverse_ps_list

doubly_robust_ate = np.nanmean(doubly_robust_list)
doubly_robust_list.append(doubly_robust_ate)
results_dict['doubly_robust'] = doubly_robust_list

print(stratification_ate)
print(linear_regression_ate)
print(non_linear_regression_ate)
print(inverse_ps_ate)
print(doubly_robust_ate)

#esults = pd.DataFrame(data=results_dict)
#results.to_csv('results_%d_min.csv'%(BIN_DURATION))

0.3037976860614079
0.11805530917996435
0.0636306929995773
0.5720116373899895
-0.0126110863887734
